#Testar modelo

In [ ]:
# !pip install openai==0.28

In [ ]:
!pip install backoff

In [ ]:
!pip install openai

In [ ]:
import pandas as pd
import backoff #bilioteca para reexecutar o codigo caso ele caia em um erro determinado por nós, considerando que esse erro geralmente está relacioando a limite de taxa da API ou algo parecido
#ele espera um tempo para reexecutar
import os
from openai import OpenAI
import time
import numpy as np
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import (
    f1_score,
    precision_score,
    recall_score,
    classification_report
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sample_comments_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IC/Teste_modelos/resultado_consolidado.csv')

In [ ]:
import re
import html
def pre_processamento (x):

  retorno = re.sub(r'http(s)?://\S+|www\.\S+', '', x)
  verdadeiro = re.search(r'&\S+', retorno)

  if verdadeiro:
        retorno = html.unescape(retorno)

  return retorno

In [ ]:
sample_comments_df.loc[:, 'Comentário'] = sample_comments_df.loc[:, 'Comentário'].apply(lambda x: pre_processamento(x))

#### Loading comments

In [ ]:
sample_comments_df.shape

(171, 14)

In [ ]:
comment_ids = sample_comments_df['id'].tolist()

In [ ]:
comments_list = sample_comments_df['Comentário'].tolist()

#### GPT-4º

In [ ]:
DEFAULT_TEMPERATURE = 0 #quanto menor, maior será a precisão e menor a criatividade
DEFAULT_MAX_TOKENS = 800
DEFAULT_MODEL = "gpt-4o-2024-08-06"

In [ ]:
client = OpenAI(api_key="")

In [ ]:
INSTRUCTIONS = f"""Você é um assistente que classifica comentários do Reddit em Português do Brasil (PT-BR) como "Positivo", "Negativo" ou "Neutro".
Você receberá o texto de um comentário e a sua tarefa é classificar o sentimento do texto fornecido. Use somente as informações abaixo para fazer a predição.

1. Para cada comentario se limite a escolher apenas uma dessas três opções, sem acrescentar texto explicativo e sem marcar outras opções que não sejam uma dessas três: "Positivo", "Negativo" ou "Neutro";
2. Marque somente como "Positivo" os comentários que tiver certeza, alta confiança de que tenham o sentimento positivo;
3. Marque somente como "Negativo" os comentários que tiver certeza, alta confiança de que tenham o sentimento negativo;
4. Marque somente como "Neutro" os comentários que tiver certeza, alta confiança de que tenham o sentimento neutro.
""".strip() #O método .strip() remove espaços em branco (incluindo espaços, tabulações, quebras de linha, etc.) do início e do final de uma string.
#o "\" define uma quebra de linha para melhor visualização, mas não signifca quebra de linha na string em si

In [ ]:
# @backoff.on_exception(backoff.expo, openai.error.RateLimitError)
def send_prompt(prompt: str, model: str = DEFAULT_MODEL) -> str:

    messages = [
        {"role": "system", "content": INSTRUCTIONS},
        {"role": "user", "content": prompt}
    ]

    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=DEFAULT_TEMPERATURE,
        max_tokens=DEFAULT_MAX_TOKENS
    )

    answer = resp.choices[0].message.content.strip()

    return answer

In [ ]:
len(comment_ids), len(comments_list)

(171, 171)

In [ ]:
predictions_sample = []

In [ ]:
processed_ids = {}

In [ ]:
len(processed_ids)

0

In [ ]:
%%time

c = 1
for c_id, comment in zip(comment_ids, comments_list):
    if c_id in set(processed_ids):
        continue
    prompt = f"Comentário: {comment}"

    answer = send_prompt(prompt=prompt)

    if c % 10 == 0 :
        print(f"collected predictions for {c} comments...")
        time.sleep(20)

    predictions_sample.append({
        'id': c_id,
        'text': comment,
        'prediction': answer
    })

    c += 1

collected predictions for 10 comments...
collected predictions for 20 comments...
collected predictions for 30 comments...
collected predictions for 40 comments...
collected predictions for 50 comments...
collected predictions for 60 comments...
collected predictions for 70 comments...
collected predictions for 80 comments...
collected predictions for 90 comments...
collected predictions for 100 comments...
collected predictions for 110 comments...
collected predictions for 120 comments...
collected predictions for 130 comments...
collected predictions for 140 comments...
collected predictions for 150 comments...
collected predictions for 160 comments...
collected predictions for 170 comments...
CPU times: user 3.68 s, sys: 294 ms, total: 3.97 s
Wall time: 7min 7s


In [ ]:
predictions_df = pd.DataFrame(predictions_sample)

In [ ]:
processed_ids = predictions_df['id'].unique().tolist()

In [ ]:
predictions_df.head()

,id,text,prediction
0,htfz11y,Outra sugestão: procure o perfil no Instagram ...,Neutro
1,hub93yq,Campeonato de quarto,Neutro
2,hv1xsjp,Não entendi,Neutro
3,hvsbpyg,"Mano, o Lucas Lima não esperou nem 10 minutos ...",Negativo
4,hwzr3zr,Eu acho que ouvi um 🐄muuul,Neutro


In [ ]:
predictions_df.shape

(171, 3)

In [ ]:
predictions_df['prediction'].value_counts(normalize=True)

,proportion
prediction,
Negativo,0.497076
Neutro,0.397661
Positivo,0.105263


In [ ]:
predictions_df[predictions_df['prediction'] == "Não sei dizer."]

,id,text,prediction


In [ ]:
def rotulacao(text):
  if text == 'Negativo':
    return 0
  elif text == 'Positivo':
    return 2
  elif text == 'Neutro':
    return 1
  else:
    return 10


In [ ]:
predictions_df.head(3)

,id,text,prediction
0,htfz11y,Outra sugestão: procure o perfil no Instagram ...,Neutro
1,hub93yq,Campeonato de quarto,Neutro
2,hv1xsjp,Não entendi,Neutro


In [ ]:
predictions_df.loc[:, 'prediction'] = predictions_df.loc[:,'prediction'].apply(lambda x: rotulacao(x))
predictions_df.head(3)

,id,text,prediction
0,htfz11y,Outra sugestão: procure o perfil no Instagram ...,1
1,hub93yq,Campeonato de quarto,1
2,hv1xsjp,Não entendi,1


In [ ]:
merge = pd.merge(sample_comments_df, predictions_df, how='inner', on='id')
#merge.rename(columns={"prediction": "predicao_gpt"}, inplace=True)
merge = merge[['id',	'Comentário',	'class_result',	'total_agreement', 'prediction']]
merge

,id,Comentário,class_result,total_agreement,prediction
0,htfz11y,Outra sugestão: procure o perfil no Instagram ...,2,1,1
1,hub93yq,Campeonato de quarto,1,1,1
2,hv1xsjp,Não entendi,1,1,1
3,hvsbpyg,"Mano, o Lucas Lima não esperou nem 10 minutos ...",0,1,0
4,hwzr3zr,Eu acho que ouvi um 🐄muuul,2,0,1
...,...,...,...,...,...
166,iwyyax0,1 O que fizeram em Los Angeles foi outra cois...,0,0,1
167,iyvd0es,"A última que eu tomei foi no meio desse ano, e...",2,1,2
168,izqc5yj,Lógico que desabou kkkkk,1,0,1
169,j0kxoll,como é bom ter democracia,2,1,2


In [ ]:
from sklearn.metrics import (
    f1_score,
    precision_score,
    recall_score,
    classification_report
)
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(list(merge.loc[:, "class_result"]), list(merge.loc[:, "gpt_prompt1"]))

0.7602339181286549

In [ ]:
f1_score(list(merge.loc[:, "class_result"]), list(merge.loc[:, "gpt_prompt1"]), average='weighted')

0.7621811894848011

In [ ]:
precision_score(list(merge.loc[:, "class_result"]), list(merge.loc[:, "gpt_prompt1"]), average='weighted')

0.8109735122119022

In [ ]:
recall_score(list(merge.loc[:, "class_result"]), list(merge.loc[:, "gpt_prompt1"]), average='weighted')

0.7602339181286549

In [ ]:
len(merge[merge['prediction'] == merge['class_result']])

130

In [ ]:
merge = merge.rename(columns={'prediction': 'gpt_prompt1'})
merge

,id,Comentário,class_result,total_agreement,gpt_prompt1
0,htfz11y,Outra sugestão: procure o perfil no Instagram ...,2,1,1
1,hub93yq,Campeonato de quarto,1,1,1
2,hv1xsjp,Não entendi,1,1,1
3,hvsbpyg,"Mano, o Lucas Lima não esperou nem 10 minutos ...",0,1,0
4,hwzr3zr,Eu acho que ouvi um 🐄muuul,2,0,1
...,...,...,...,...,...
166,iwyyax0,1 O que fizeram em Los Angeles foi outra cois...,0,0,1
167,iyvd0es,"A última que eu tomei foi no meio desse ano, e...",2,1,2
168,izqc5yj,Lógico que desabou kkkkk,1,0,1
169,j0kxoll,como é bom ter democracia,2,1,2


In [ ]:
merge.to_csv('/content/drive/MyDrive/Colab Notebooks/IC/Teste_modelos/resultado_consolidado.csv')